# `DSML Workshop 05` - Hands on Data Science with real-world geotagged data

During this workshop, we will do some hands-on data science with real-world geotagged data. We will dive into an excerpt of Car2Go data from the city of Stuttgart (STR). You can find the dataset on ILIAS. We will focus on how to engineer features based on the available data. At the end, we will also get into some more advanced analyses.

We will go through the following:

- **Task**: Visualizing the Iris dataset
- **Task**: Getting started with feature engineering
- **Feature Engineering Part 1:** Working with datetime data
- **Feature Engineering Part 2**: Merge additional data
- **Feature Engineering Part 3**: Working with geotagged data
- **Descriptive analyses**: Hands-on data science using carsharing data

---

## 1. Task: Visualizing the Iris dataset

Last week, we learned how to visualize data using Matplotlib and Seaborn. Let us put what we learned together using a familiar dataset: the iris flower dataset.

Complete the following steps:
1. Read in the `iris.csv` dataset and drop missing data.
2. Create a simple scatter plot of the "number" and "Petal.Width" features. Annotate your graph appropriately.
3. You may have noticed some striking patterns emerging from this first plot. So let's dig deeper. Plot Petal.Width vs. Petal.Length on a scatter plot. Colour-code the different species. Hint: You could run `plt.scatter()` for each species and use different colors to differentiate between the species.
4. Finally, let us plot a histogram - a common plot type in data science - which shows the number of occurences of a given observation in the sample. **Use the Seaborn library for this**. Plot the Petal.Width feature. Hint: Using the `data=`, `x=` and `hue=` parameters of the `sns.histplot()` function can be very helpful here.

In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt

# read data (it is in the same directory, so no need to specify the file path)
iris = pd.read_csv("iris.csv")

# fill missing values using bfill or ffill
iris = iris.dropna().copy()
iris.info()

In [ ]:
# first, let's create a figure object
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(10,6))

# define plot
ax.scatter(iris["number"], iris["Petal.Width"])

# add axis and title labels
ax.set_xlabel("Flower ID")
ax.set_ylabel("Petal Width (cm)")
ax.set_title("Petal Width per Flower")

# show figure
plt.show()

In [ ]:
# define species list and corresponding colors
species = list(iris["Species"].unique())
colors = ["r","g","b"]

# create fig object
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(10,6))

# iterate over species and plot
for i in range(3):
    x = iris[iris["Species"] == species[i]]["Petal.Length"]
    y = iris[iris["Species"] == species[i]]["Petal.Width"]
    ax.scatter(x,y, c="white", edgecolors=colors[i], marker="o", label=species[i])

# add labels, title and legend
ax.set_xlabel("Petal Length")
ax.set_ylabel("Petal Width")
ax.set_title("Petal length & width per iris flower")
ax.legend()

plt.show()

In [ ]:
# use seaborn to create histogram
import seaborn as sns
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(10,6))
sns.histplot(data=iris, x="Petal.Width", bins=20, hue="Species", ax=ax, palette="magma", edgecolor="white")
plt.show()

---

## 2. Task: Getting started with feature engineering

This week, we will look at what features we can derive from the data we have available. We will use a large carsharing dataset during the workshop.

Among other things, we will work with temporal data. This means that our dataframe contains information like "2017-11-01 00:10:01". Brainstorm features that you could extract from this piece of information and collect them in the list below.

Feature list:
- date
- year
- month
- day of year
- day of month
- day of week
- is_weekday
- is_holiday
- hour
- minute
- ...

Let's create a dataframe containing hypothetical date values. Execute the following cell.

In [ ]:
import pandas as pd

test_data = pd.DataFrame({"timestamp": ["2024-05-01", "2024-05-02", "2024-05-03", "2024-05-04", "2024-05-05", "2024-05-06", "2024-05-07"]})

test_data

Manually look up the day of the week (Monday, Tuesday, Wednesday,...) for the given dates and add them in a second column called "weekday_manually".

In [ ]:
test_data["weekday_manually"] = ["Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Monday", "Tuesday"]

test_data

This is, of course, not feasible for large datasets. Luckily, Pandas has extensive built-in functionalities to deal with datetime data. Use `pd.to_datetime(test_data["timestamp"], format="%Y-%m-%d")` to transform the timestamp column into datetime format. Check the data type of timestamp using the .info() method.

In [ ]:
test_data["timestamp"] = pd.to_datetime(test_data["timestamp"], format="%Y-%m-%d")

test_data.info()

Finally, let's again retrieve the name of the weekday, but this time Python should do the work for us. Simply use the `.dt.day_name()` method on the timestamp feature and save the result to a new column called "weekday_pandas".

In [ ]:
test_data["weekday_pandas"] = test_data["timestamp"].dt.day_name()

test_data

---

## 3. Feature Engineering Part 1: Working with datetime data

To keep size manageable we have uploaded a 2-week data snippet of Car2Go movement data from the city of Stuttgart (STR) to ILIAS. Download the file from ILIAS and save it in the WS_05_Feature_engineering folder. We use data from November, 2017. In the interest of time we have done a lot of the pre-processing for you. Simply execute the two cells below to read in the data in the desired format.

In [ ]:
# import all relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
sns.set_style("white")
sns.set_palette("GnBu_d")

# also import these "new" libraries 
# Note: you may have to download and add them to your environment (using e.g. 'conda install -c conda-forge folium')
import folium
from folium import plugins
from folium.plugins import HeatMap
import geopandas as gpd
from datetime import date, time, datetime, timedelta
import math
import random

In [ ]:
# read in data
STR_Nov = pd.read_csv("Car2Go_STR_Nov.csv", encoding = "ISO-8859-1")

# sort values by car ID and timestamp
STR_Nov.sort_values(["car ID","timestamp"], inplace=True)

# inspect data
STR_Nov.head(10)

In [ ]:
# familiarize yourself with the data
STR_Nov.info()

In [ ]:
# familiarize yourself with the data
STR_Nov.describe()

### Datetime in Python, Numpy and Pandas

In this first part, we want to focus on the timestamp column. For this type of data, Python provides specialized tools such as the built-in libraries `datetime` and `dateutil` or NumPy's `datetime64`. You can read up on these here: https://docs.python.org/3/library/datetime.html

- `datetime.date`: An idealized naive date, assuming the current Gregorian calendar always was, and always will be, in effect. Attributes: year, month, and day.

- `datetime.time`: An idealized time, independent of any particular day, assuming that every day has exactly `24*60*60` seconds (there is no notion of “leap seconds” here). Attributes: hour, minute, second, microsecond, and tzinfo.

- `datetime.datetime`: A combination of a date and a time. Attributes: year, month, day, hour, minute, second, microsecond, and tzinfo.

- `datetime.timedelta`: A duration expressing the difference between two date, time, or datetime instances to microsecond resolution.

- `datetime.tzinfo`: An abstract base class for time zone information objects. These are used by the datetime and time classes to provide a customizable notion of time adjustment (for example, to account for time zone and/or daylight saving time).

- `datetime.timezone`: A class that implements the tzinfo abstract base class as a fixed offset from the UTC.

To get familiar working with datetime data, let's first create two hypothetical datetime variables and perform some operations on them.

In [ ]:
# let's define two datetime objects
date1 = datetime(year=2020, month=5, day=13, hour=12, minute= 29, second=59, microsecond=109262)
date2 = datetime(year=2020, month=5, day=14, hour=16, minute= 1, second=59, microsecond=109262)

# verify that the type is datetime
type(date1)

In [ ]:
date1

If I subtract date2 from date1, the result is a timedelta object.

In [ ]:
# subtract date2 from date1
delta = date2 - date1

# verify that the type is timedelta
type(delta)

In [ ]:
delta

A powerful extension of `DateTime` is the `dateutil` module which allows you to parse date and time information that comes in different formats.

In [ ]:
from dateutil import parser
date3 = parser.parse("13th of May, 2020")
date4 = parser.parse("13/05/2020")

print(date3)
print(date4)

The power of datetime and dateutil lie in their flexibility and easy syntax: you can use these objects and their built-in methods to easily perform nearly any operation you might be interested in. They have their weaknesses when you wish to work with large arrays of dates and times: just as lists of Python numerical variables are suboptimal compared to NumPy-style typed numerical arrays, lists of Python datetime objects are suboptimal compared to typed arrays of encoded dates. Let's look at how we can perform vectorized datetime operations using numpy.

In [ ]:
# create a date object in numpy format
date5 = np.array('2019-05-07', dtype=np.datetime64)
date5

In [ ]:
# once we have a date object in numpy format, we can perform vectorized operations on it
date5-np.arange(12)

In this workshop we will focus on the datetime functionality provided by Pandas. Pandas builds upon all the tools above to provide a `Timestamp` object, which combines the ease-of-use of datetime with the efficient storage and vectorized interface of `numpy.datetime64`. From a group of these Timestamp objects, Pandas can construct a `DatetimeIndex` that can be used to index data in a Series or DataFrame; we'll see how this works below.

Pandas was developed in the context of financial modeling, so as you might expect, it contains a fairly extensive set of tools for working with dates, times, and time-indexed data. Date and time data comes in a few forms, which we will discuss here:
* __Time stamps__ reference particular moments in time (e.g., May 7th, 2019 at 4:00pm).
* __Time intervals__ and periods reference a length of time between a particular beginning and end point; for example, the year 2015. Periods usually reference a special case of time intervals in which each interval is of uniform length and does not overlap (e.g., 24 hour-long periods comprising days).
* __Time deltas__ or durations reference an exact length of time (e.g., a duration of 22.56 seconds).

In the following we will introduce how to work with each of these types of date/time data in Python/Pandas. This short section is by no means a complete guide to the time series tools available in Python or Pandas, but will equip you with the relevat methods and techniques necessary to complete your team assignments. For more information on this please consult the relevant Pandas documentation [here](http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html).

Let's return to our Car2Go data.

In [ ]:
# remember our dataset
STR_Nov.head()

In [ ]:
# let's transform the timestamp column to datetime
STR_Nov["timestamp"] = pd.to_datetime(STR_Nov["timestamp"], format="%Y-%m-%d %H:%M:%S") # you need to specify the correct format; more info here: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

# check out the data type of timestamp now
STR_Nov.info()

### Indexing based on timestamps

Let's suppose we only want to look at Car2Go data of a certain time period on a single day - let's say we are interested in data between 6am and 8pm on 10th November 2017. Since Pandas has a built-in datetime functionality we can perform boolean operations on the dates, which allows us to specify the desired interval through masking.

In [ ]:
# define start and end datetime variables
start = datetime(year=2017, month=11, day=10, hour=6)
end = datetime(year=2017, month=11, day=10, hour=20)

# create Car2Go availability dataframe for selected period through masking
Car2Go_single_day = STR_Nov[(STR_Nov["timestamp"]>=start)&(STR_Nov["timestamp"]<=end)].copy()

# check out the number of observations during the selected period
len(Car2Go_single_day)

### Aggregating based on timestamps

One common need for time series data is resampling at a higher or lower frequency. This can be done using the `resample()` method. The types of frequency or date offsets that can be selected are shown in the below table. By adding an S suffix, instead of the end of a period, the start is selected.

| Code      | Description       | Code        | Description                |
|-----------|-------------------|-------------|----------------------------|
| ``D``     | Calendar day      | ``B``       | Business day               |
| ``W``     | Weekly            |             |                            |
| ``ME/MS`` | Month end/start   | ``BME/BMS`` | Business month end/start   |
| ``QE/QS`` | Quarter end/start | ``BQE/BQS`` | Business quarter end/start |
| ``YE/YS`` | Year end/start    | ``BYE/BYS`` | Business year end/start    |
| ``h``     | Hours             | ``bh``      | Business hours             |
| ``min``   | Minutes           |             |                            |
| ``s``     | Seconds           |             |                            |
| ``ms``    | Milliseconds      |             |                            |
| ``us``    | Microseconds      |             |                            |
| ``ns``    | Nanoseconds       |             |                            |

Let's say we are interested in the charging levels of each car for our selected time period. If we wanted to get the average charging level by car, we could use a familiar function, namely groupby.

In [ ]:
# get average charging level by car
Car2Go_single_day[["car ID", "tank/charge level"]].groupby("car ID").mean()

This information might not really be useful, because it is not granular enough. However, our original dataframe gives us the charging levels for each available car every 5 minutes, which might be too detailed. What about getting the average charging level for every car on an hourly basis? The resample method can help us here. For the resample method to work, we have to make timestamp the index of our dataframe.

In [ ]:
# select only relevant columns
Car2Go_single_day = Car2Go_single_day[["timestamp", "car ID", "tank/charge level"]].copy()

# make timestamp index
Car2Go_single_day = Car2Go_single_day.set_index("timestamp")

# group by car and resample to hourly frequency
Car2Go_single_day.groupby("car ID").resample("h").mean()

### Temporal Feature Extraction

Finally, we want to extract some features from our timestamp. You may be surprised how much information is contained in this single feature, which might carry significant explanatory value for the dependent variable, as is the case in many real-world settings. Pandas makes it easy for us to extract temporal features from a timestamp using the dt accessor. Below are some examples.

In [ ]:
# extract the date
STR_Nov["date"] = STR_Nov["timestamp"].dt.date

# extract the time
STR_Nov["time"] = STR_Nov["timestamp"].dt.time

# extract year
STR_Nov["year"] = STR_Nov["timestamp"].dt.year

# extract month
STR_Nov["month"] = STR_Nov["timestamp"].dt.month

# extract day
STR_Nov["day"] = STR_Nov["timestamp"].dt.day

# extract hour
STR_Nov["hour"] = STR_Nov["timestamp"].dt.hour

# extract weekday (0=Monday, 6=Sunday)
STR_Nov["weekday"] = STR_Nov["timestamp"].dt.weekday

# check out the data
STR_Nov.head()

**Question**: What type of data is the `weekday` feature? Nominal, ordinal, interval or a ratio? Can it be used like this in a regression model? Why?

To get the `weekday` feature into a format which is usable in regression models we use a technique which is often referred to as one-hot-encoding. In essence we use a binary variable for each weekday and drop one.

In [ ]:
# implement one-hot-encoding
STR_Nov[["Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]] = pd.get_dummies(STR_Nov["weekday"],drop_first=True)

# inspect final dataframe
STR_Nov

---

## 4. Feature Engineering Part 2: Merge additional data

You might have interesting information from other sources that you want to include in your analysis. First, let's pretend that Car2Go provides us with a second dataset that contains information on the type of cars in their fleet.

In [ ]:
# first, we generate our hypothetical car model data
brands = ["Fiat", "BMW", "Volkswagen", "Ford", "Volvo", "Tesla", "Porsche"]
seats = [2,3,4,5]
color = ["grey", "black", "blue", "red", "green"]

car_models = pd.DataFrame({"car ID": STR_Nov["car ID"].unique()})
car_models["brand"] = np.random.choice(brands, len(car_models))
car_models["seats"] = np.random.choice(seats, len(car_models))
car_models["color"] = np.random.choice(color, len(car_models))

car_models.head()

In [ ]:
# next, we merge the car model data with our main dataframe
STR_Nov_cars = STR_Nov.merge(car_models, how = "left", left_on = "car ID", right_on = "car ID")

STR_Nov_cars.head()

Second, let's pretend that we have information on the urban traffic congestion in Stuttgart. Specifically, we have information on the average additional time spent during a 10km car trip in the inner city for every hour.

In [ ]:
# first, we generate our hypothetical congestion data
congestion = pd.DataFrame({'timestamp': pd.date_range('2017-11-01', '2017-11-15', freq="h", inclusive='left')})
congestion["congestion"] = np.random.randint(0,20, len(congestion))

congestion

In [ ]:
# next, we merge the congestion data with our main dataframe
# note that in contrast to the car model data, we do not have any column that exactly matches in both dataframes
# for merges on datetime, pd.merge_asof comes to the rescue

# for this to work, both dataframes must be sorted by datetime
STR_Nov.sort_values("timestamp", inplace=True)

STR_Nov_congestion = pd.merge_asof(STR_Nov, congestion, on = "timestamp", direction = "nearest") # you could also set direction to "backward" or "forward"; what makes most sense here?

In [ ]:
# let's return to our original sorting by car ID and timestamp and inspect the data
STR_Nov_congestion.sort_values(["car ID","timestamp"], inplace=True)

STR_Nov_congestion.head(10)

**Question**: How could we perform the merge based on hour using the normal merge function? Hint: think about the temporal features we extracted.

---

## 5. Feature Engineering Part 3: Working with geotagged data

The dataset contains geospatial data. Therefore, we can generate interesting features from the location information. To do that, we first convert our dataframe to a geopandas dataframe. A geopandas dataframe is like a pandas dataframe that allows for geographical operations on so called geometry columns. In our dataframe, we have latitude and longitude data. We can transform our dataframe to a geopandas dataframe and create the geometry column from these coordinates.

**Note**: this might go beyond what you will be able to do within your group projects. Nevertheless, it is very useful to to get a first understanding of how to work with geospatial data, as it enables you to conduct really cool analyses!

In [ ]:
# create geopandas dataframe; do not worry about the crs parameter for now, we will get to this in a minute
str_nov_geo = gpd.GeoDataFrame(STR_Nov, geometry=gpd.points_from_xy(STR_Nov['long'], STR_Nov['lat']), crs="epsg:4326")

str_nov_geo.head()

Notice the difference between the dataframes before and after: there is a new column called geometry. This is the column that enables the geopandas operations on this dataframe. One example: we can now easily plot the locations using .plot().

In [ ]:
# let's plot the locations of a single car - the one with car ID WME4513901K820402
str_nov_geo[str_nov_geo["car ID"] == "WME4513901K820402"].plot()
plt.show()

Note that you cannot really see anything because we are missing a map in the background. We will look at more advanced plots later on.

Now let's create a second geo dataframe with only a single observation: the Stuttgart Central Station, which is located at latitude 48.783889 and longitude 9.181667.

In [ ]:
# create normal dataframe with single observation
STR_central_station = pd.DataFrame({"name": ["stuttgart central station"], "longitude": [9.181667], "latitude": [48.783889]})

# transform into geodataframe
STR_central_station = gpd.GeoDataFrame(STR_central_station, geometry=gpd.points_from_xy(STR_central_station["longitude"],STR_central_station["latitude"]), crs="epsg:4326")

STR_central_station

Let's plot the location of the central station together with our car with ID WME4513901K820402.

In [ ]:
fig, ax = plt.subplots()
str_nov_geo[str_nov_geo["car ID"] == "WME4513901K820402"].plot(ax=ax, color="lightblue")
STR_central_station.plot(ax=ax, color="red")
plt.show()

What might be a really useful feature is the distance of any car to the central station. Calculating distances is actually really easy with geopandas.

In [ ]:
str_nov_geo.distance(STR_central_station.iloc[0]["geometry"])

**But wait!** Look at the warning - there seems to be something wrong with our distance calculation. This is related to the so-called Coordinate Reference System (CRS) we are using here. Our geometries are defined using latitudes and longitudes, following the most commmonly used CRS named WGS84 (https://epsg.io/4326; this is what we specified using the crs parameter when creating the geodataframes before). Two problems:
- The unit of the CRS is degrees. Does it make sense to calculate distances between degrees? How to interpret these?
- Our world is spherical (surprise!), which makes it hard to represent it on a 2D map. Any so-called "projection" introduces distortion (head over to https://www.thetruesize.com for some mind-blowing visualizations to get the point) - but there are projections that are accurate for certain parts of the world.

A nice website for looking up CRSs is https://epsg.io. One such projected system that gives coordinates in meters is Europe UTM. UTM is a projection that is split into 'slices'. Each slice is valid only for a part of the earth, which depends on the location you want to model. In our case the suitable system has the code 25837 (https://epsg.io/25837). We can easily transform our coordinates in both dataframes using `to_crs()`.

In [ ]:
# change CRSs
str_nov_geo = str_nov_geo.to_crs(epsg=25837)
STR_central_station = STR_central_station.to_crs(epsg=25837)

# check using .crs attribute
STR_central_station.crs

In [ ]:
# check out new geometries
STR_central_station

Now the geometries look strange, don't they? Actually, they are more useful now. The values you see there are the meters this point is away from (0,0). Now you can't really tell where on earth these locations are without knowing where (0,0) is, but you can calculate the distance using the euclidean formula! Lets put this to use.

In [ ]:
str_nov_geo["distance_to_central_station"] = str_nov_geo.distance(STR_central_station.iloc[0]["geometry"])

str_nov_geo.head()

Another nice feature one can generate is to check whether an observation is in the proximity of another location. For example, I might not be interested in the exact distance, but just want to know if a car is within a radius of 500 meters from the central station. I can easily do this as shown below.

In [ ]:
# change the geometry of STR_central_station by adding a buffer of 500 meters
STR_central_station["geometry"] = STR_central_station.buffer(500)

STR_central_station

In [ ]:
# check whether the Car2Go locations are within the circle around the central station
str_nov_geo['close_to_central_station'] = str_nov_geo.within(STR_central_station.iloc[0]["geometry"])

str_nov_geo.head()

---

## 6. Descriptive analyses: Hands-on data science using carsharing data

Let's see some examples of what information we can get from our dataset.

### Fleet size and availability

In [ ]:
# first, let's explore fleet size
fleet_size = len(STR_Nov["car ID"].unique())

print("The STR fleet consists of {} cars (for the observed time frame)".format(fleet_size))

Let's explore fleet availability throughout the week and day by checking how many unique cars are available for each day and hour of day.

In [ ]:
# Availability over time (5 min intervals)
STR_Nov_Date_avail = STR_Nov.groupby("timestamp")["car ID"].nunique() # this counts unique vehicles per 5-min time interval

fig,ax = plt.subplots(figsize=(16,9)) 

ax.plot(STR_Nov_Date_avail)
ax.set_title("Cars available per time interval",fontsize=16)

plt.show()

Next we perform some aggregations. We are interested in typical availability
- by day of week
- by hour

In [ ]:
# Availability by day of week (0 = Monday, 6 = Sunday)

STR_Nov_weekday_avail = STR_Nov.groupby(["date","weekday"])["car ID"].nunique()
STR_Nov_weekday_avail = pd.DataFrame(STR_Nov_weekday_avail)
STR_Nov_weekday_avail

In [ ]:
fig,ax = plt.subplots(figsize=(10,4)) 

sns.boxplot(x=STR_Nov_weekday_avail.index.get_level_values(1), y=STR_Nov_weekday_avail["car ID"],ax=ax)
plt.show()

In [ ]:
# Availability by time of day

STR_Nov_time_avail = STR_Nov.groupby(["date","hour"])["car ID"].nunique()
STR_Nov_time_avail = pd.DataFrame(STR_Nov_time_avail)

In [ ]:
fig,ax = plt.subplots(figsize=(10,4)) 

sns.boxplot(x=STR_Nov_time_avail.index.get_level_values(1), y=STR_Nov_time_avail["car ID"],ax=ax)
plt.show()

#### Question: What do you observe? <br>

- Describe the patterns of car availability shown in the descriptive graphs above. Which graphs are particularly instructive?
- What are your assumptions regarding the causes of the observed patterns?

### Car Movement & Location Intelligence

How can we infer car movements from this data? Let's have a look at what the data provides. A lot of what follows below is a demonstation of possible geospatial analytics tasks, which could be highly relevant from a fleet operator's point of view (Where is the demand? Where should my units be at what time?)

**Technical note**: for this tutorial we use `folium` for visualization (see [here](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data) for a quick introduction).

Let us check where cars are typically located by plotting a heat map of locations for a given hour of the day.

In [ ]:
# to work with folium, we first zip the latitude and longitude data
STR_Nov["Coordinates"] = list(zip(STR_Nov["lat"].round(4),STR_Nov["long"].round(4)))

# For performance reasons we select a single car only - pick randomly from the list
car_list = STR_Nov["car ID"].unique()
selected_car_ID = random.choice(car_list)
print(selected_car_ID)

In [ ]:
# define new map
STR_heat_map = folium.Map(location=(48.76507, 9.25371),tiles='OpenStreetMap',zoom_start=12, control_scale=True, max_zoom=20)

# add heat map
STR_heat_map.add_child(plugins.HeatMap(STR_Nov[STR_Nov["car ID"]==selected_car_ID]["Coordinates"], radius=20))

STR_heat_map

We now wish to identify the movement of the selected car. To do this, we draw the location points and trajectories.

In [ ]:
# define empty map
STR_map_points = folium.Map(location=(48.76507, 9.25371),  tiles='OpenStreetMap', 
                       zoom_start=10, control_scale=True, max_zoom=20)


# draw positions for selected car
STR_Nov_car_select = STR_Nov[STR_Nov["car ID"]==selected_car_ID]

positions = list(STR_Nov_car_select["Coordinates"].unique())
for position in positions:
    folium.CircleMarker(radius=10, location=position, popup='The Waterfront', 
                                 color='crimson', fill_color='crimson').add_to(STR_map_points)
    
STR_map_points

We also add the trajectories to get an understanding of the distance traveled. Do you see any problem with such an approach?

In [ ]:
STR_map_trajectory = folium.Map(location=(48.76507, 9.25371),  tiles='OpenStreetMap', 
                       zoom_start=11, control_scale=True, max_zoom=20)

# draw trajectory for a selected car
folium.PolyLine(STR_Nov_car_select["Coordinates"],color="blue", weight=5, opacity=0.8).add_to(STR_map_trajectory)
STR_map_trajectory

#### Question: Where do you see limitations and how can we deal with them?

- What are the key limitations of this data granularity?
- What are approaches to infering distance traveled?
- What are approaches to infering the exact route traveled?

### Using the fleet as a Virtual Power Plant (VPP)

Note that in STR the full active car fleet is electric and therefore constitutes a potential distributed energy resource (DER) that can be used to provide flexibility. The fleet consists of Smart EQ fortwos and Mercedes Benz A Class models, each with a battery of size 17.6 kWh and charging rates of 6 kW (22kW optional). We can use this battery capacity to provide flexibility to the power grid (remember the example from the first lecture?).

First, let's look at where the charging points are located.

From the data we can also infer where charging points must be located. Let's have a look...

In [ ]:
# How many charging locations are there?
print("Number of charging points:", len(list(STR_Nov[STR_Nov["charging"]==1]["Coordinates"].unique())))

In [ ]:
# define dataset for plotting, i.e. charging_pos
charging_pos=list(STR_Nov[STR_Nov["charging"]==1]["Coordinates"].unique())

# define empty map
STR_map_charging = folium.Map(location=(48.76507, 9.25371),  tiles='OpenStreetMap', 
                       zoom_start=11, control_scale=True, max_zoom=20)


# draw positions of charging positions
for charge_point in charging_pos:
    folium.CircleMarker(radius=3, location=charge_point, popup='The Waterfront', 
                                 color='green', fill_color='green').add_to(STR_map_charging)
    
STR_map_charging

We might be interested in how much capacity we have available at any given point in time, which we could use to trade in the ancillary market. For this we first need to define how many cars are charging at a particular instance. Remember the following technical information regarding the fleet:

- Battery capacity: 17.6 kWh
- Charging rate: 6 kW
- Optional fast charging: 22 kW

In [ ]:
Battery_energy_capa = 17.6 #kWh
Charging_rate = 6 #kW
Charging_rate_fast = 22 #kW
Max_EV_range = 200 #km

In [ ]:
# cars connected at a given point in time (we can simply sum the charging dummies)
STR_Nov_charging = STR_Nov.groupby("timestamp")["charging"].sum()

fig,ax = plt.subplots(figsize=(16,9)) 

ax.plot(STR_Nov_charging)

ax.set_title("# cars connected at a given point in time")
ax.set_ylabel("# of cars")
ax.set_xlabel("Date")

plt.show()

In [ ]:
# Let us look at trends over the day - How many cars get connected per hour per day?
STR_Nov_charging_tod = pd.DataFrame(STR_Nov[STR_Nov["charging"]==1]).groupby(["date","hour"])["car ID"].nunique()
STR_Nov_charging_tod = pd.DataFrame(STR_Nov_charging_tod)

STR_Nov_charging_tod.head(10)

In [ ]:
fig,ax = plt.subplots(figsize=(16,9)) 

sns.boxplot(x=STR_Nov_charging_tod.index.get_level_values(1), y=STR_Nov_charging_tod["car ID"],ax=ax)
ax.set_title("# cars connected by time of day (hour intervals)")
plt.show()

In [ ]:
# after knowing the number of cars (= number of batteries) connected, let's transform this into available capacity
fig, ax = plt.subplots(figsize=(16,9), ncols=2, nrows=1, sharex=True, sharey=True)

STR_Nov_capa_normal = STR_Nov.groupby("timestamp")["charging"].sum()*Charging_rate
STR_Nov_capa_fast = STR_Nov.groupby("timestamp")["charging"].sum()*Charging_rate_fast

# Normal speed______
ax[0].set_title("Capacity per 5 min slot with 6kW charging [kW]")
ax[0].plot(STR_Nov_capa_normal, "--b")

# fast speed_______
ax[1].set_title("Capacity per 5 min slot with 22kW charging [kW]")
ax[1].plot(STR_Nov_capa_fast, "--g")

plt.show()

In the ancillary services market you pledge capacity (in kW) for which you get reimbursed. If you get called (either for upwards or downwards regulation) you need to feed in or consume at this rate (or below). Therefore, as a plant operator, you need to demonstrate that you will be able to sustain the pledged load for the full 15min time interval. Consequently, we must not forget about the battery capacities and states of charge of our vehicles.

To verify, we explore the state of charge across time for the cars that are charging as this will give us an indication for the energy that is available for upwards and downwards regulation (i.e. the time frame for which the VPP can be called) - an important constraint!

In [ ]:
# connected battery capacity (in kWh)

# What do we have available for downwards regulation (i.e. how much energy can be absorbed?)
STR_Nov["Connected free Battery Capacity [kWh]"] = ((100-STR_Nov["tank/charge level"])/100)*Battery_energy_capa*STR_Nov["charging"]

# What do we have available for upwards regulation (i.e. how much energy can be fed back into the grid?)
STR_Nov["Connected SOC [kWh]"] = ((STR_Nov["tank/charge level"])/100)*Battery_energy_capa*STR_Nov["charging"]

STR_Nov.head()

In [ ]:
Connected_downwards_energy = STR_Nov[STR_Nov["charging"]==1].groupby("timestamp")["Connected free Battery Capacity [kWh]"].sum()
Connected_upwards_energy = STR_Nov[STR_Nov["charging"]==1].groupby("timestamp")["Connected SOC [kWh]"].sum()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), ncols=2, nrows=1, sharex=True, sharey=True)

# downwards capacity
ax[0].set_title("Downwards energy [kWh]")
ax[0].plot(Connected_downwards_energy, "--b")

# upwards capacity
ax[1].set_title("Upwards energy [kWh]")
ax[1].plot(Connected_upwards_energy, "--g")

plt.show()

#### Questions

- What patterns do you see? Why? Do you see scope for building a strong predictive model?
- How do you evaluate the case of using the VPP in the ancillary services markets (secondary reserve) (compare fast charging and normal charging case)?
- Recall the mechanism of the secondary reserve market: participants use bids for a capacity price for every 15min. From inspecting the graph above and comparing with the available capacity from above, is there a relevant state of charge constraint? Is it a problem? How big of a problem is it?
- How does this constraint influence the capacity that can be pledged in the capacity markets?

---